In [31]:
import openai
import yaml
from tqdm import tqdm
openai.organization = "org-KamOJqC0ZbZBYPx5caHVk8VJ"
openai.api_key = "add_apikey_here"

def generate_fuzzing_codes(prompt):
    resp = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a helpful assistant who will generate syntactically correct Ansible YAML Playbook for testing purposes. Do not include any explanation or context or introduction statement. Make sure to include ``` before and after Ansible code to denote the YAML section"},
                {"role": "user", "content": prompt}
            ]
        )

    return resp


In [32]:
answer = generate_fuzzing_codes('Create a YAML Ansible playbook to exhibit the following issue: Module invocation had junk after the JSON data: Last login: Mon Jan')
answer['choices'][0]

<OpenAIObject at 0x7fca1241e6b0> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "message": {
    "content": "```\n---\n- name: Display module invocation error\n  hosts: localhost\n  tasks:\n    - debug:\n        msg: \"Module invocation had junk after the JSON data: Last login: Mon Jan\"\n```",
    "role": "assistant"
  }
}

In [33]:
import pandas as pd
df = pd.read_feather('../github/github_issues.feather')


In [34]:
df.head()

,IssueID,IssueTitle,IssueBody,CreationTimestamp,CommentCount
0,80249,"community.general.lxd ""url: https://"" [Errno 2...",### Summary\n\nI'm trying to list remote hosts...,2023-03-17T14:12:47Z,2
1,80230,Integer variable value has trailing zero trunc...,### Summary\r\n\r\nI have 2 vars files contain...,2023-03-15T14:45:46Z,4
2,80228,rescue module did not work when in handlers,"### Summary\r\n\r\nI used block in handlers ,t...",2023-03-15T07:04:45Z,2
3,80199,lineinfile append to string replaces whole line,### Summary\n\nIf you want to append a string ...,2023-03-13T09:53:35Z,4
4,80193,Unable to run dnf or yum modules in Python3 ve...,### Summary\n\nI am trying to run a playbook i...,2023-03-10T19:07:00Z,5


In [35]:
df['Prompt'] = "Create a YAML Ansible playbook to exhibit the following issue: " + df['IssueTitle']

In [36]:
df['Output'] = ''
df = df[:100]

In [37]:
count = 0
generation_limit = 100
list_of_responses = []

In [38]:
with tqdm(total=None, desc="Generating playbooks") as pbar:
    for ind in df.index:
        if count >= generation_limit:
            break
        if df.iloc[ind]['Output'] == '':
            response = generate_fuzzing_codes(df.iloc[ind]['Prompt'])
            response = response['choices'][0]['message']['content']
            list_of_responses.append((ind, response))
            df.iloc[ind, 6] = response
            pbar.update(1)
            count += 1


Generating playbooks: 100it [13:49,  8.30s/it]


In [13]:
df.iloc[1, 6]

"Here is an Ansible YAML Playbook to demonstrate the issue where integer variable value has trailing zero truncated:\n\n```\n- name: Integer variable value has trailing zero truncated\n  hosts: localhost\n  gather_facts: false\n\n  vars:\n    number: 10.50\n\n  tasks:\n    - name: Display number variable\n      debug:\n        var: number\n```\n\nIn the above playbook, we have defined a variable called 'number' with a value of 10.50. However, when we display this variable using the debug module, Ansible truncates the trailing zero and displays the value as 10.5. This demonstrates the issue where integer variable value has trailing zero truncated."

In [14]:
df.iloc[1, 6].split("```")[1]

'\n- name: Integer variable value has trailing zero truncated\n  hosts: localhost\n  gather_facts: false\n\n  vars:\n    number: 10.50\n\n  tasks:\n    - name: Display number variable\n      debug:\n        var: number\n'

In [39]:
print("Number of playbooks created:", (df.Output != '').sum())
df.iloc[0]['Output']

Number of playbooks created: 100


'```\n---\n- name: Test Playbook\n  hosts: localhost\n  gather_facts: no\n\n  tasks:\n  - name: Execute community.general.lxd\n    community.general.lxd:\n      url: https://\n    ignore_errors: yes\n```\n'

In [40]:
import yaml
def get_yaml_data(text):
    try:
        data = text.split("```")[1]
        code = yaml.safe_load(data)
        #print("Valid YAML syntax")
        return 1
    except:
        #print("Invalid YAML syntax:", error)
        code = ''
        return 0

In [41]:
df['Valid'] = df['Output'].apply(get_yaml_data)

In [42]:
df['Valid']

0     1
1     1
2     1
3     1
4     1
     ..
95    1
96    1
97    1
98    1
99    1
Name: Valid, Length: 100, dtype: int64

In [43]:
df['Valid'].sum()

91

In [52]:
def remove_tilda(text):
    return text.split('```')[1]

In [53]:
df['Code'] =df['Output'].apply(remove_tilda)

In [47]:
df.to_feather('chatgpt_github_yaml.feather')

In [54]:
df.to_csv('chatgpt_github_yaml.csv')

In [55]:
import pandas as pd
ndf = pd.read_feather('chatgpt_github_yaml.feather')

In [57]:
ndf

,IssueID,IssueTitle,IssueBody,CreationTimestamp,CommentCount,Prompt,Output,Valid
0,80249,"community.general.lxd ""url: https://"" [Errno 2...",### Summary\n\nI'm trying to list remote hosts...,2023-03-17T14:12:47Z,2,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: Test Playbook\n hosts: loca...,1
1,80230,Integer variable value has trailing zero trunc...,### Summary\r\n\r\nI have 2 vars files contain...,2023-03-15T14:45:46Z,4,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: Exhibit integer variable iss...,1
2,80228,rescue module did not work when in handlers,"### Summary\r\n\r\nI used block in handlers ,t...",2023-03-15T07:04:45Z,2,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- hosts: localhost\n gather_facts: ...,1
3,80199,lineinfile append to string replaces whole line,### Summary\n\nIf you want to append a string ...,2023-03-13T09:53:35Z,4,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: An example playbook to exhib...,1
4,80193,Unable to run dnf or yum modules in Python3 ve...,### Summary\n\nI am trying to run a playbook i...,2023-03-10T19:07:00Z,5,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: Test playbook for dnf/yum mo...,1
...,...,...,...,...,...,...,...,...
95,79676,AttributeError: 'int' object has no attribute ...,### Summary\n\nRunning a playbook :https://dpa...,2023-01-05T22:28:43Z,9,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: Testing AttributeError \n h...,1
96,79665,shebang doesn't use ansible_python_interpreter,### Summary\r\n\r\nhttps://github.com/ansible/...,2023-01-04T21:40:27Z,9,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: Display ansible_python_inter...,1
97,79661,package_facts requires python-rpm on SUSE syst...,### Summary\n\nThis bug report is identical to...,2023-01-04T13:08:06Z,3,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: Example playbook to exhibit ...,1
98,79646,ansible-test disallows an underscore as a vari...,### Summary\r\n\r\nI have noticed that pylint ...,2023-01-02T18:50:00Z,3,Create a YAML Ansible playbook to exhibit the ...,```\n---\n- name: Testing ansible-test\n host...,1


In [58]:
ndf['IssueBody'] = ndf['IssueBody'].str.slice(stop=30000)

In [ ]:
ndf.to_csv('chatgpt_ndf.csv')